In [1]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pandas as pd
from sklearn import svm
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from sklearn.model_selection import train_test_split
from sklearn import datasets
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, LeaveOneOut, cross_val_predict
from sklearn.model_selection import KFold  
from sklearn.linear_model import LogisticRegression  
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn import svm  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
import math
import datetime
import multiprocessing as mp
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

import random


# In[25]:


# cores = mp.cpu_count()
# pool = mp.Pool(processes=20)

# min1=min(list(X_standard.shape))
# a1=list(range(10,min1,10))
# a1[-1]=min1
# pcc_top1=pool.map(find_best_pca_lr,a1)  ##（SVM，LR）
# pool.terminate()


# a1=pd.DataFrame()
# for i1 in pcc_top1:
#     a1=pd.concat([a1,i1], axis=0)
# a1.columns=['tr_TP', 'tr_TN', 'tr_FP', 'tr_FN', 'tr_Sen', 'tr_Spe', 'tr_Acc', 'tr_PPV', 'tr_NPV', 'tr_MCC', 'tr_AUC','TP',
#        'TN', 'FP', 'FN', 'Sen', 'Spe', 'Acc', 'PPV', 'NPV', 'MCC', 'AUC','PCA']   

# a1=a1.sort_values(['Acc','tr_Acc','PCA'], ascending=[False,False,True])

In [2]:
def read_single_csv(input_path,i):

    df_chunk=pd.read_csv(input_path,
                         chunksize=10000,
#                          sep='\t'
                        sep=i
                        )
    res_chunk=[]
    for chunk in df_chunk:
        res_chunk.append(chunk)
    res_df=pd.concat(res_chunk)
    return res_df

In [3]:
def get_measures_gridloo(label, score):
    label = np.array(label)
    score = np.array(score)
    
    N  = len(label)
    TP = sum((label == 1) & (score == 1))
    TN = sum((label == 0) & (score == 0))
    FP = sum((label == 0) & (score == 1))
    FN = sum((label == 1) & (score == 0))

    # init all measures to nan
    measures = {measure: float("nan") for measure in ("Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC")}
    
    measures["TP"] = TP
    measures["TN"] = TN
    measures["FP"] = FP
    measures["FN"] = FN
    
    S = (TP + FN) / N
    P = (TP + FP) / N

    if (TP + FN) > 0:
        measures["Sen"] = round(TP/(TP+FN), 4)

    if (TN + FP) > 0:
        measures["Spe"] = round(TN/(TN+FP), 4)

    if (TP + FP + FN + TN) > 0:
        measures["Acc"] = round((TP+TN)/(TP+FP+FN+TN), 4)

    if (TP + FP) > 0:
        measures["PPV"] = round(TP/(TP+FP), 4)

    if (TN + FN) > 0:
        measures["NPV"] = round(TN/(TN+FN), 4)

    if (S*P*(1-S)*(1-P)) > 0:
        measures["MCC"] = round((TP/N - S*P)/(math.sqrt(S*P*(1-S)*(1-P))), 4)
    
    
    measures["AUC"]= roc_auc_score(label, score)
    return pd.DataFrame([measures],
                        columns=["TP", "TN", "FP",
                                 "FN", "Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC"])

In [4]:
##SVM
def find_best_pca(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    scoring = {'AUC': 'roc_auc'}
    cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
    parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 
    grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
    grid_search.fit(pca_X_train, train_y)
    
    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [5]:
def find_best_pca_lr(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

    param_grid = {'penalty':['l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],  
              "solver":["newton-cg", "lbfgs","liblinear"],
#             "C":list(np.arange(0,0.1,0.005)),   
#               "solver":["newton-cg", "lbfgs","liblinear"]
            }
    LR_grid = LogisticRegression()
    grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=9)
    grid_search.fit(pca_X_train, train_y)


    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
   

    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [9]:
cancer='HNSC'
path='D:\\lymph_meta_xmiseq\\tcga_data\\'
# e1="./"+cancer+"_delta_pcc_wilcox.csv"
# delta_pcc=read_single_csv(e1,',')
# delta_pcc=delta_pcc.drop(['Unnamed: 0','pair'],axis=1)
# delta_pcc=delta_pcc.dropna(axis=0,how='any')

# result=delta_pcc.T
# train=result.iloc[:,0:-1].values.astype('float')
# target=result.iloc[:,-1].values.astype('float')

train=pd.read_csv(path+cancer+"\\admat_final.csv").iloc[:,2:].T.values
target=pd.read_csv(path+cancer+"\\gctm_label.csv",index_col=0).values.ravel()

In [10]:
train

array([[ 8.61302310e-03, -1.67276214e-02, -7.71436140e-02, ...,
         1.49324996e-02, -9.55921388e-04,  1.64712682e-04],
       [-1.01694133e-02,  5.04230888e-02,  2.35664766e-01, ...,
         1.22894564e-02, -8.40306508e-03, -6.14264990e-02],
       [ 1.36782686e-02,  3.05819334e-02,  1.00714180e-02, ...,
        -2.37681151e-02, -9.66026000e-05, -8.76877865e-03],
       ...,
       [ 2.17807588e-03, -2.54476766e-03,  3.25767040e-03, ...,
         1.49324996e-02,  2.65327542e-03, -3.56168709e-04],
       [ 3.56776436e-03, -7.91398304e-04, -2.99311432e-02, ...,
         7.51862011e-03,  9.02637160e-03, -4.25091257e-03],
       [ 1.23089540e-02,  1.73426352e-02, -4.00936485e-02, ...,
         1.49324996e-02,  9.02637160e-03,  8.16723247e-03]])

In [11]:
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int64)

In [183]:
train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=10
                                                   )

standardScaler = StandardScaler()
standardScaler.fit(train_X)
X_standard = standardScaler.transform(train_X)
X_standard_test = standardScaler.transform(test_X)

n_components=0.95
estimator = PCA(n_components=n_components,random_state=10)
pca_X_train = estimator.fit_transform(X_standard)
pca_X_test = estimator.transform(X_standard_test)
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

In [184]:
train_X.shape

(126, 2281)

In [185]:
#svm
scoring = {'AUC': 'roc_auc'}
cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

svc_grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
svc_grid_search.fit(pca_X_train, train_y)

#LR
param_grid = {'penalty':['l1', 'l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
#               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
            }
LR_grid = LogisticRegression(max_iter=1000)
LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
LR_grid_search.fit(pca_X_train, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.45230769        nan 0.52369231 0.54769231
 0.54769231 0.72153846 0.76123077 0.74523077        nan        nan
 0.45230769        nan 0.50030769 0.75292308 0.75292308 0.75292308
 0.79230769 0.75292308        nan        nan 0.45230769        nan
 0.52369231 0.848      0.848      0.82430769 0.864      0.832
        nan        nan 0.68246154        nan 0.75292308 0.888
 0.888      0.87230769 0.88       0.88              nan        nan
 0.92              nan 0.92       0.88061538 0.88061538 0.85661538
 0.86461538 0.86461538        nan        nan 0.944             nan
 0.936      0.84061538 0.84061538 0.86461538 0.86461538 0.87261538
        nan        nan 0.944             nan 0.87261538 0.84861538
 0.84861538 0.86461538 0.86461538 0.87261538        nan        nan
 0.90430769        nan 0.87261538 0.85661538 0.85661538 0.84861

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=10, shuffle=True),
             estimator=LogisticRegression(max_iter=1000), n_jobs=10,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [186]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]
clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
clf.fit(pca_X_train, train_y)

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LinearSVC(C=5), n_jobs=10)

In [187]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]

param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                 LogisticRegression(C=0.1),LogisticRegression(C=1),LogisticRegression(C=10),LogisticRegression(C=100),
                                 LogisticRegression(C=1000)]}

Stacking_grid =StackingClassifier(estimators=estimators,)
Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
Stacking_grid_search.fit(pca_X_train, train_y)
Stacking_grid_search.best_estimator_

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LogisticRegression(C=100))

In [188]:
train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components

test_pre_y = Stacking_grid_search.predict(pca_X_test)
test_res1=get_measures_gridloo(test_y,test_pre_y)

In [189]:
print(train_res1)
print(test_res1)

   TP  TN  FP  FN     Sen     Spe     Acc     PPV     NPV     MCC       AUC
0  64  50   7   5  0.9275  0.8772  0.9048  0.9014  0.9091  0.8076  0.902365
   TP  TN  FP  FN  Sen  Spe  Acc  PPV  NPV  MCC  AUC
0  30  24   0   0  1.0  1.0  1.0  1.0  1.0  1.0  1.0


In [ ]:
%%time
################random seed loop#################
train_scores=[]
test_scores=[]
loopn=20
scoring='roc_auc'
np.random.seed(42)
random_states=np.random.choice(range(101), loopn, replace=False)
print(random_states)
for i in range(loopn):
    train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=random_states[i]
                                                   )

    standardScaler = StandardScaler()
    standardScaler.fit(train_X)
    X_standard = standardScaler.transform(train_X)
    X_standard_test = standardScaler.transform(test_X)
    #calculate max n_components
    estimator = PCA(n_components=0.95,random_state=random_states[i])
    pca_X_train = estimator.fit_transform(X_standard)

    #n_components=0.95
    n_components=range(10,min(pca_X_train.shape),10)
    #n_components=[0.99,0.95,0.90,0.85]
    best_pca_train_scores=[]
    best_pca_test_scores=[]
    for j in n_components:
        estimator = PCA(n_components=j,random_state=random_states[i])
        pca_X_train = estimator.fit_transform(X_standard)
        pca_X_test = estimator.transform(X_standard_test)
        cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_states[i]) 
    
        #scoring = {'AUC': 'roc_auc'}
        cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
        gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
        parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
        #cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_states[i]) 

        svc_grid_search=GridSearchCV(estimator=SVC(random_state=random_states[i]),param_grid=parameters,cv=cvx,scoring=scoring)
        svc_grid_search.fit(pca_X_train, train_y)

        param_grid = {'penalty':['l1', 'l2'],
                      "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
                      "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
    #               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
                    }
        LR_grid = LogisticRegression(max_iter=1000)
        LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
        LR_grid_search.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]
        clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
        clf.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]

        param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                         LogisticRegression(C=0.1),LogisticRegression(C=1),LogisticRegression(C=10),LogisticRegression(C=100),
                                         LogisticRegression(C=1000)]}

        Stacking_grid =StackingClassifier(estimators=estimators,)
        Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
        Stacking_grid_search.fit(pca_X_train, train_y)
        Stacking_grid_search.best_estimator_

        train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
        train_res1=get_measures_gridloo(train_y,train_pre_y)

        test_pre_y = Stacking_grid_search.predict(pca_X_test)
        test_res1=get_measures_gridloo(test_y,test_pre_y)
        
        best_pca_train_scores.append(train_res1.loc[:,"AUC"])
        best_pca_test_scores.append(test_res1.loc[:,"AUC"])
    
    train_scores.append(np.max(best_pca_train_scores))
    test_scores.append(best_pca_test_scores[np.argmax(best_pca_train_scores)])

[84 55 66 67 45 39 22 44 10  0 18 30 97 33 77  4 93 78 12 31]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.86860639
 0.86860639 0.87574925 0.87574925 0.87574925        nan        nan
 0.5               nan 0.5        0.89662005 0.89662005 0.90019148
 0.90019148 0.90019148        nan        nan 0.5               nan
 0.5        0.91812354 0.91812354 0.91758242 0.91563437 0.91303696
        nan        nan 0.89314019        nan 0.88105228 0.9258991
 0.9258991  0.92992008 0.93540626 0.93378288        nan        nan
 0.91410256        nan 0.91280386 0.9534632  0.9534632  0.94285714
 0.94417249 0.94157509        nan        nan 0.94816017        nan
 0.94415584 0.95595238 0.95595238 0.95205628 0.94805195 0.94545455
        nan        nan 0.94816017        nan 0.94545455 0.95075758
 0.95075758 0.94816017 0.94805195 0.94545455        nan        nan
 0.94816017        nan 0.94545455 0.94945887 0.9494588

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.88452381
 0.88452381 0.88324176 0.88324176 0.88324176        nan        nan
 0.5               nan 0.5        0.91683317 0.91683317 0.91802364
 0.91802364 0.91802364        nan        nan 0.5               nan
 0.5        0.95894106 0.95894106 0.95899101 0.95637696 0.95639361
        nan        nan 0.90304695        nan 0.87470862 0.9961372
 0.9961372  0.99094239 0.99224109 0.99094239        nan        nan
 0.95901598        nan 0.96280386 0.99483849 0.99483849 0.99353979
 0.99353979 0.99353979        nan        nan 0.97559108        nan
 0.98704629 0.99227439 0.99227439 0.98967699 0.99353979 0.99482184
        nan        nan 0.95729271        nan 0.99482184 0.98581419
 0.98581419 0.98321678 0.99225774 0.99482184        nan        nan
 0.96122211        nan 0.99482184 0.98066933 0.9806693

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.88904429
 0.88904429 0.89045122 0.89045122 0.89045122        nan        nan
 0.5               nan 0.5        0.91087246 0.91087246 0.91467699
 0.91337829 0.91467699        nan        nan 0.5               nan
 0.5        0.95227273 0.95227273 0.94851815 0.94970862 0.94851815
        nan        nan 0.88399101        nan 0.85944056 0.98083583
 0.98083583 0.9757992  0.97708125 0.9757992         nan        nan
 0.95660173        nan 0.94925907 0.98976856 0.98976856 0.98986014
 0.98986014 0.98986014        nan        nan 0.96068931        nan
 0.97984515 0.98848651 0.98848651 0.98995171 0.99244089 0.99115884
        nan        nan 0.97457542        nan 0.99126707 0.98460706
 0.98460706 0.98618049 0.98987679 0.99117549        nan        nan
 0.97871295        nan 0.99117549 0.98330836 0.983308

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.89290709
 0.89290709 0.89301532 0.89301532 0.89301532        nan        nan
 0.5               nan 0.5        0.90838328 0.90838328 0.91606727
 0.91476856 0.91606727        nan        nan 0.5               nan
 0.5        0.95754246 0.95754246 0.953663   0.95734266 0.9549617
        nan        nan 0.88399101        nan 0.86073926 0.99123377
 0.99123377 0.98499001 0.99004329 0.98628871        nan        nan
 0.96918082        nan 0.9742674  0.99253247 0.99253247 0.98744589
 0.98874459 0.98874459        nan        nan 0.98982684        nan
 0.98733766 0.99123377 0.99123377 0.98636364 0.98874459 0.98874459
        nan        nan 0.98354978        nan 0.99004329 0.98993506
 0.99123377 0.98636364 0.98744589 0.98874459        nan        nan
 0.9728355         nan 0.99004329 0.98874459 0.9887445

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

In [21]:
print('mean')
print(np.mean(train_scores))
print(np.mean(test_scores))
print("std")
print(np.std(train_scores))
print(np.std(test_scores))

mean
0.9554157131960335
0.9131250000000002
std
0.013808098372004751
0.030370255688017454


In [66]:
print(train_scores)
print(test_scores)

[0.7990713381173491, 0.8402279442802871, 0.845082313212326, 0.8347931616715915, 0.7996517517940058, 0.8505170958210215, 0.8268256648374841, 0.8656606162937949, 0.8274060785141409, 0.7942169691853102, 0.8499366821443648, 0.8039257070493878, 0.8353735753482483, 0.8153756859434359, 0.8093604896580836, 0.8147952722667792, 0.8062473617560152, 0.8390671169269734, 0.8613866610384129, 0.8722562262558043]
[0    0.82735
Name: AUC, dtype: float64, 0    0.744444
Name: AUC, dtype: float64, 0    0.851282
Name: AUC, dtype: float64, 0    0.85641
Name: AUC, dtype: float64, 0    0.824786
Name: AUC, dtype: float64, 0    0.747863
Name: AUC, dtype: float64, 0    0.802564
Name: AUC, dtype: float64, 0    0.736752
Name: AUC, dtype: float64, 0    0.770085
Name: AUC, dtype: float64, 0    0.788034
Name: AUC, dtype: float64, 0    0.850427
Name: AUC, dtype: float64, 0    0.821368
Name: AUC, dtype: float64, 0    0.755556
Name: AUC, dtype: float64, 0    0.85812
Name: AUC, dtype: float64, 0    0.811966
Name: AUC, dty